In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
from functools import reduce
import os
%matplotlib inline
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')

PROJECT_ROOT_DIR = "C:/Users/signa/_Relkat/"
#STEP = "data_selection"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "plots")

def save_fig(fig_id, tight_layout=True, fig_extension="pdf", resolution=500):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution) 

In [ ]:
date = "28.04"
orig_folder = 'C:/Users/signa/_Relkat/data/data_csv/'+date+"/"
dest_folder = 'C:/Users/signa/_Relkat/data/data_csv/'+date+"/plots"
print(date)

In [ ]:
MD = pd.read_csv(os.path.join(orig_folder,'MD_Temperaturen_'+date+'.csv'),index_col = 0)
print('loading',os.path.join(orig_folder,'MD_Temperaturen_'+date+'.csv'))
print('number of data points: ',MD.shape[0])
MD.reset_index(drop=True,inplace=True)

In [ ]:
pd.set_option('display.max_rows', 700)
MD.drop(['uuid','plc_time','sequence_number','value_string','value_bytes','identifier'],axis=1,inplace=True)

MD['start_cycle'] = np.datetime64('NaT')
MD['end_cycle'] = np.datetime64('NaT')
MD['end_cycle'] = np.where((MD['label'].str.contains('Taktzeit 1 Start '))&(MD['value_bool']==False),MD['sys_time'],np.datetime64('NaT'))
MD.loc[(MD['value_bool']==True),"start_cycle"] = MD['sys_time']
MD['cycle'] = MD['value_bool'][~MD['start_cycle'].isnull()]*np.array([i for i in range(MD['start_cycle'].nunique())])
MD.sort_values(by=['sys_time'],inplace=True)
MD['cycle'].fillna(method='ffill',inplace=True)

# MD['sys_time'] = MD['sys_time'].astype('datetime64[s]')

In [ ]:
features = ['Kühlaggregat: Themperatur Rücklauf Maschine in °C (DB_PROZESSWERTE.iw_130_92 )','Kühlaggregat: Themperatur Vorlauf Maschine in °C (DB_PROZESSWERTE.iw_130_91 )']
features

In [ ]:
MD.head(10)

In [ ]:
df_features = []
for feature in features:
    df_feature = MD[MD['label']==feature][['sys_time','value_integer','value_double']]
    df_feature[feature] = np.where(df_feature['value_integer'].isnull(),df_feature['value_double'],df_feature['value_integer'])
    df_feature.drop(['value_integer','value_double'],axis=1,inplace=True)
    df_features.append(df_feature)

In [ ]:
df_MD = reduce(lambda left,right: pd.merge(left,right,on=['sys_time'],how='outer'), df_features)

In [ ]:
df_MD.head()

In [ ]:
pd.set_option('display.max_rows', 1000)
df_MD.set_index('sys_time',inplace=True)
df_MD = df_MD.resample('s').mean()
df_MD.reset_index(inplace=True,drop=False)
df_MD.head(300)

In [ ]:
MD = MD[['sys_time','cycle']]
MD.drop_duplicates(subset='sys_time',inplace=True)
MD.reset_index(drop=True,inplace=True)
df_MD = pd.merge(MD,df_MD, on='sys_time',how='inner')
df_MD

In [ ]:
MD_2 = pd.read_csv(os.path.join(orig_folder,'MD_'+date+'.csv'),index_col = 0)
print('loading',os.path.join(orig_folder,'MD_'+date+'.csv'))
print('number of data points: ',MD_2.shape[0])
MD_2.reset_index(drop=True,inplace=True)

In [ ]:
pd.set_option('display.max_rows', 700)
MD_2.drop(['uuid','plc_time','sequence_number','value_string','value_bytes','identifier'],axis=1,inplace=True)
MD_2['sys_time'] = MD_2['sys_time'].apply(lambda x: pd.to_datetime(x))
MD_2['sys_time'] = MD_2['sys_time'].apply(lambda x: x.tz_localize(None))
MD_2.sort_values(by=['sys_time'],inplace=True)
MD_2.reset_index(inplace=True,drop=True)
MD_2

In [ ]:
features_2 = list(MD_2['label'].unique())[1:]
features_2 = ['(DB_KÜHLMITTEL.Sollwert )',
 'Hydraulik: externe Sollwert-Vorgabe (mow_210_01 )',
 '(DB_KÜHLMITTEL_HOCHDRUCK.Sollwert )']

In [ ]:
df_features_2 = []
for feature_2 in features_2:
    df_feature_2 = MD_2[MD_2['label']==feature_2][['sys_time','value_integer','value_double']]
    df_feature_2[feature_2] = np.where(df_feature_2['value_integer'].isnull(),df_feature_2['value_double'],df_feature_2['value_integer'])
    df_feature_2.drop(['value_integer','value_double'],axis=1,inplace=True)
    df_features_2.append(df_feature_2)

In [ ]:
df_MD_2 = reduce(lambda left,right: pd.merge(left,right,on=['sys_time'],how='outer'), df_features_2)

In [ ]:
pd.set_option('display.max_rows', 1000)
df_MD_2.head(1000)

In [ ]:
pd.set_option('display.max_rows', 1000)
df_MD_2.set_index('sys_time',inplace=True)
df_MD_2 = df_MD_2.resample('s').mean()
df_MD_2.reset_index(inplace=True,drop=False)
df_MD_2.head(300)

In [ ]:
df_MD_3 = pd.merge(df_MD,df_MD_2, on='sys_time',how='outer')
df_MD_3.head(200)

In [ ]:
MD_3 = pd.read_csv(os.path.join(orig_folder,'MD_Durchfluss_'+date+'.csv'),index_col = 0)
print('loading',os.path.join(orig_folder,'MD_Durchfluss_'+date+'.csv'))
print('number of data points: ',MD_3.shape[0])
MD_3.reset_index(drop=True,inplace=True)

In [ ]:
pd.set_option('display.max_rows', 700)
MD_3.drop(['uuid','plc_time','sequence_number','value_string','value_bytes','identifier'],axis=1,inplace=True)
MD_3['sys_time'] = MD_3['sys_time'].apply(lambda x: pd.to_datetime(x))
MD_3['sys_time'] = MD_3['sys_time'].apply(lambda x: x.tz_localize(None))
MD_3['sys_time'] = MD_3['sys_time'].astype('datetime64[s]')
MD_3.sort_values(by=['sys_time'],inplace=True)
MD_3.head(10)

In [ ]:
#features_3 = list(MD_3['label'].unique())
#MD_3.reset_index(drop=True,inplace=True)
features_3 = ['Kühlschmierstoff: Durchfluss ND Vorlauf  in l/min (DB_PROZESSWERTE.iw_450_93 )','Kühlschmierstoff: Durchfluss HD Vorlauf in l/min (DB_PROZESSWERTE.iw_450_92 )']

In [ ]:
df_features_3 = []
for feature_3 in features_3:
    df_feature_3 = MD_3[MD_3['label']==feature_3][['sys_time','value_integer','value_double']]
    df_feature_3[feature_3] = np.where(df_feature_3['value_integer'].isnull(),df_feature_3['value_double'],df_feature_3['value_integer'])
    df_feature_3.drop(['value_integer','value_double'],axis=1,inplace=True)
    df_features_3.append(df_feature_3)

In [ ]:
df_MD_4 = reduce(lambda left,right: pd.merge(left,right,on=['sys_time'],how='outer'), df_features_3)

In [ ]:
pd.set_option('display.max_rows', 1000)
df_MD_4.set_index('sys_time',inplace=True)
df_MD_4 = df_MD_4.resample('s').mean()
df_MD_4.head(300)

In [ ]:
pd.set_option('display.max_rows', 1000)
print(df_MD_5.shape)
df_MD_5[10000:11000]

In [ ]:
df_MD_5 = pd.merge(df_MD_3,df_MD_4, on='sys_time',how='outer')
df_MD_5['cycle'].fillna(method='ffill',inplace=True)
df_MD_5['cycle'] = df_MD_5['cycle'].astype(int)

In [ ]:
df_MD_5['Temperatur_Differenz_Kühlaggregat_Vorlauf_Rücklauf in °C'] = df_MD_5['Kühlaggregat: Themperatur Rücklauf Maschine in °C (DB_PROZESSWERTE.iw_130_92 )'] - df_MD_5['Kühlaggregat: Themperatur Vorlauf Maschine in °C (DB_PROZESSWERTE.iw_130_91 )']
df_MD_5.head(200)

In [ ]:
df_MD_5.to_csv(os.path.join(orig_folder,'feature_table_'+date+'.csv'))

In [ ]:
features = list(df_MD_5.columns[2:-3])
#df_MD_5['Kühlaggregat: Themperatur Rücklauf Maschine (mib_130_92 )'].hist()
features

In [ ]:
MD_feature_table = pd.read_csv(os.path.join(orig_folder,'feature_table_'+date+'.csv'))

In [ ]:
def format_axis(ax, color):    
    for t in ax.get_yticklabels():
        t.set_color(color)
    ax.yaxis.label.set_color(color)
    return None

def plotFeat(df,cycle,feature_list,ax,offset):    
      
    df_grouped = df[df['cycle']== cycle].reset_index() 
      
    format_axis(ax, 'k')     
    
    colors = ['darkorange','gold','lime','deeppink','midnightblue','red','black','blue','dodgerblue']
    
    for i in range(len(feature_list)):
        feature = df_grouped[feature_list[i]]
        ax.plot(feature,  color=colors[i+offset], marker='o',label = feature_list[i])   

    return ax 

In [ ]:
def aggregateFeat(df,cycle,feature_list):      
      
    fig, axes = plt.subplots(nrows=2,ncols=1,figsize=(36, 40))
    plt.title('Cycle: {0} '.format(cycle),fontsize=40)
    ax1 = axes[0]
    ax1.set_title('Temperaturverläufe Versuche 28.4.2021', fontsize=40)
    ax1.set_xlabel('time/sec',fontsize=30)
    ax1.set_ylabel('°C/min',fontsize=30)
    ax1.tick_params(axis='x', labelsize=25)
    ax1.tick_params(axis='y', labelsize=25) 
    
    Hyd,KSS,KSS_HD = np.array(df[df['cycle']==cycle]['Hydraulik: externe Sollwert-Vorgabe (mow_210_01 )'])[0],\
                     np.array(df[df['cycle']==cycle]['(DB_KÜHLMITTEL.Sollwert )'])[0],\
                     np.array(df[df['cycle']==cycle]['(DB_KÜHLMITTEL_HOCHDRUCK.Sollwert )'])[0]
    
    delta_max  = np.array(df[df['cycle']==cycle]['Temperatur_Differenz_Kühlaggregat_Vorlauf_Rücklauf in °C']).max()
    delta_mean = np.array(df[df['cycle']==cycle]['Temperatur_Differenz_Kühlaggregat_Vorlauf_Rücklauf in °C']).mean()
    
    textstr = r'cycle: ' ' %.0f\nStellgrößen\nHyd: %.0f\nKSS: %.0f\nKSS_HD: %.0f\nδ Kühlaggregat max: %.2f\nδ Kühlaggregat mean: %.2f'\
        %(cycle,Hyd,KSS,KSS_HD,delta_max,delta_mean)
    ax1.text(12,12,textstr,fontsize=35, 
    bbox={'facecolor':'gray', 'alpha':0.1, 'pad':10})
                           
    plotFeat(df,cycle,feature_list[:7],ax1,offset=0)  

    handles, labels = ax1.get_legend_handles_labels()
    ax1.legend(handles, labels,prop={'size': 23})
   
    ax2 = axes[1]
    ax2.set_title('Durchflussgrößen Versuche 28.4.2021', fontsize=40)
    ax2.set_xlabel('time/ sec',fontsize=30)
    ax2.set_ylabel('l / min',fontsize=30)
    ax2.tick_params(axis='x', labelsize=25)
    ax2.tick_params(axis='y', labelsize=25) 
        
    
    plotFeat(df,cycle,feature_list[7:],ax2,offset=7)  

    handles, labels = ax2.get_legend_handles_labels()
    ax2.legend(handles, labels,prop={'size': 23})    
   
    
    return

In [ ]:
#######ENTER CYCLE###############################################################################################

cycle = 62

feature_1  = 'Kühlschmierstoff: Temperatur Vorlauf in °C (DB_PROZESSWERTE.iw_450_90 )' #
feature_2  = 'Kühlschmierstoff: Temperatur Spänebehälter in °C (DB_PROZESSWERTE.iw_450_91 )'#
feature_3  = 'Temperatur_Differenz_Kühlaggregat_Vorlauf_Rücklauf in °C'


feature_4  = 'Kühlaggregat: Themperatur Vorlauf Maschine in °C (DB_PROZESSWERTE.iw_130_91 )'
feature_5  = 'Kühlaggregat: Themperatur Rücklauf Maschine in °C (DB_PROZESSWERTE.iw_130_92 )'

feature_6  = 'Maschinenraum: Temperatur in °C (DB_PROZESSWERTE.iw_577_02 )'
feature_7  = 'Hydraulik: Temperatur in °C (DB_PROZESSWERTE.iw_210_01 )'

feature_8  = 'Kühlschmierstoff: Durchfluss ND Vorlauf  in l/min (DB_PROZESSWERTE.iw_450_93 )'
feature_9 = 'Kühlschmierstoff: Durchfluss HD Vorlauf in l/min (DB_PROZESSWERTE.iw_450_92 )'


feature_list = [feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9]
fig = aggregateFeat(df_MD_5,cycle,feature_list)
#save_fig('Features for cycle_{0} '.format(cycle))

In [ ]:
fig = plt.figure(figsize=(25, 14))
ax1 = plt.subplot(5,1,1)
plt.xticks(np.arange(0,93,1))
plt.plot(d_agg_en['cycle'],d_agg_en['max'], label="Max. Differenz in °C Kühlaggregat",color='red')
plt.plot(d_agg_en['cycle'],d_agg_en['mean'],label="Mittlere Differenz in °C Kühlaggregat",color='orange')
plt.legend(fontsize=20)
ax1.set_xlabel('Takt',fontsize=20)

ax2 = plt.subplot(5,1,2,sharex=ax1)
plt.xticks(np.arange(0,93,1))
plt.plot(d_agg_en['cycle'],d_agg_en['energy_total'], label="Summe Energieverbrauch in kWh",color='mediumblue')
plt.legend(fontsize=20)

ax3 = plt.subplot(5,1,3,sharex=ax2)
plt.xticks(np.arange(0,93,1))
plt.plot(d_agg_en['cycle'],d_agg_en['Hyd'], label="Hyd",color='black')
plt.legend(fontsize=20)

ax4 = plt.subplot(5,1,4,sharex=ax3)
plt.xticks(np.arange(0,93,1))
plt.plot(d_agg_en['cycle'],d_agg_en['KSS'], label="KSS",color='darkslategray')
plt.legend(fontsize=20)

ax5 = plt.subplot(5,1,5,sharex=ax4)
plt.xticks(np.arange(0,93,1))
plt.plot(d_agg_en['cycle'],d_agg_en['KSS_HD'], label="KSS_HD",color='gray')
plt.legend(fontsize=20)
fig = save_fig('Temperaturdifferenz_Kuehlaggregat_VL_RL_28.04.2021')

In [ ]:
d_agg = df_MD_5.groupby(['cycle'])['Temperatur_Differenz_Kühlaggregat_Vorlauf_Rücklauf in °C'].aggregate([max,np.mean])
d_agg.reset_index(inplace=True)

In [ ]:
# data_28_4 = pd.read_csv('C:/Users/signa/_Relkat/data/data_csv/28.04/28.04_total_reduced.csv',index_col=0)
# data_28_4['sys_time'] = data_28_4['start_time'].astype('datetime64[s]')
# df_merge = pd.merge(data_28_4,df_MD_5,on='sys_time',how='inner')
df_en = df_merge[['cycle','energy_total','Hyd','KSS','KSS_HD']]
d_agg_en = pd.merge(d_agg,df_en,on='cycle',how='outer')
d_agg_en

In [ ]:
pd.set_option('display.max_rows', 1000)
df_merge.loc[0,'cycle']=1
df_merge.loc[7,'cycle']=11
df_merge.loc[14,'cycle']=18
df_merge.loc[15,'cycle']=19
df_merge.loc[16,'cycle']=20
df_merge.loc[20,'cycle']=24
df_merge.loc[21,'cycle']=25
df_merge.loc[22,'cycle']=26
df_merge.loc[26,'cycle']=30

df_merge.loc[38,'cycle']=42
df_merge.loc[39,'cycle']=43
df_merge.loc[42,'cycle']=46
df_merge.loc[43,'cycle']=47
df_merge.loc[44,'cycle']=48
df_merge.loc[45,'cycle']=49
df_merge.loc[52,'cycle']=58
df_merge.loc[56,'cycle']=62
df_merge.loc[58,'cycle']=64
df_merge.loc[62,'cycle']=68
df_merge.loc[69,'cycle']=75
df_merge.loc[76,'cycle']=82

In [ ]:
d_agg_en.to_csv(os.path.join(orig_folder,'d_agg_plot_'+date+'.csv'))